In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Constants
IMG_SIZE = (128, 128)  # You can adjust this
BATCH_SIZE = 32

# Paths
train_dir = r"C:/wild/dataset/train"
val_dir = r"C:/wild/dataset/val"

# Image generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess images
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)


Found 301 images belonging to 2 classes.
Found 1522 images belonging to 2 classes.


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification (real vs fake)
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)


C:\Users\Vidya\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Vidya\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 446ms/step - accuracy: 0.7310 - loss: 0.6603

C:\Users\Vidya\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 30s 3s/step - accuracy: 0.7383 - loss: 0.6488 - val_accuracy: 0.8949 - val_loss: 0.3454
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 714ms/step - accuracy: 0.8539 - loss: 0.4217 - val_accuracy: 0.8949 - val_loss: 0.3610
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 752ms/step - accuracy: 0.8036 - loss: 0.4676 - val_accuracy: 0.8949 - val_loss: 0.3685
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 730ms/step - accuracy: 0.8548 - loss: 0.4170 - val_accuracy: 0.8922 - val_loss: 0.4636
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 739ms/step - accuracy: 0.8520 - loss: 0.4238 - val_accuracy: 0.8647 - val_loss: 0.4083
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 697ms/step - accuracy: 0.8590 - loss: 0.3655 - val_accuracy: 0.8962 - val_loss: 0.2922
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 739ms/step - accuracy: 0.8660 - loss: 0.3526 - val_accuracy: 0.8962 - val_loss: 0.3153
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 791ms/step - accuracy: 0.8534 - loss: 0.3285 - val_accuracy: 0.8922 - val_lo

In [6]:
model.save("plate_classifier.h5")


In [60]:
from tensorflow.keras.models import load_model

model = load_model("plate_classifier.h5")  # or skip if model is already in memory


In [29]:
import cv2
import numpy as np

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128, 128))  # use the size from training
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Path to one cropped license plate image
image_path = "C:/Users/Vidya/Desktop/wild/cropped_plates/download2.jpg"

img = preprocess_image(image_path)
prediction = model.predict(img)[0][0]

if prediction >= 0.40:
    print(f"🚩 This Number Plate is not of RTO regulation (Confidence: {prediction:.2f})")
else:
    print(f"✅ This Number Plate is of RTO regulation and is Valid for Road Use (Confidence: {1 - prediction:.2f})")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
✅ This Number Plate is of RTO regulation and is Valid for Road Use (Confidence: 0.76)


In [61]:
model.save("models/plate_classifier.h5")


In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),  # This layer now adapts to output shape automatically

    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)                   │ (None, 62, 126, 32)         │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_15 (MaxPooling2D)      │ (None, 31, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 29, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_16 (MaxPooling2D)      │ (None, 14, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 12, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_17 (MaxPooling2D)      │ (None, 6, 14, 128)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 10752)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 128)                 │       1,376,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,469,761 (5.61 MB)

 Trainable params: 1,469,761 (5.61 MB)

 Non-trainable params: 0 (0.00 B)

In [39]:
model.add(Dense(128))


In [41]:
from tensorflow.keras.models import load_model

model = load_model("plate_classifier.h5")  # or skip this if already in memory


a different CNN



In [45]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'C:/wild/dataset/train',
    target_size=(64, 128),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    'C:/wild/dataset/val',
    target_size=(64, 128),
    batch_size=32,
    class_mode='binary'
)

# Train model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)


Found 301 images belonging to 2 classes.
Found 1522 images belonging to 2 classes.
Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 36s 3s/step - accuracy: 0.7960 - loss: 0.5494 - val_accuracy: 0.8949 - val_loss: 0.3422
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 486ms/step - accuracy: 0.8201 - loss: 0.5019 - val_accuracy: 0.8949 - val_loss: 0.3688
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 486ms/step - accuracy: 0.8170 - loss: 0.4773 - val_accuracy: 0.8949 - val_loss: 0.3289
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 442ms/step - accuracy: 0.8471 - loss: 0.4128 - val_accuracy: 0.8949 - val_loss: 0.4280
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 476ms/step - accuracy: 0.8319 - loss: 0.4208 - val_accuracy: 0.8949 - val_loss: 0.2839
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 423ms/step - accuracy: 0.8582 - loss: 0.3058 - val_accuracy: 0.8949 - val_loss: 0.2317
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 468ms/step - accuracy: 0.8731 - loss: 0.2510 - val_accuracy: 0.9106 - val_loss: 0.2183
Epoch 8/10
10/10 ━━━━━━

In [46]:
model.save("license_plate_classifier.h5")


In [66]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image

# Load a test image
img_path = "C:/Users/Vidya/Desktop/wild/cropped_plates/samplemanthan_plate0.jpg"
img = image.load_img(img_path, target_size=(64, 128))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
prediction = model.predict(img_array)[0][0]

# Visualize
plt.imshow(img)
plt.axis('off')
label = " This Number Plate is not of RTO regulation " if prediction <= 0.4 else " This Number Plate is of RTO regulation and is Valid for Road Use "
plt.title(f"Prediction: {label} /n ({prediction:.2f})")
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Vidya/Desktop/wild/cropped_plates/samplemanthan_plate0.jpg'